In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import psycopg2

#### Définition de l'accès

In [2]:
load_dotenv()

USERNAME = os.getenv("INPI_USERNAME")
PASSWORD = os.getenv("INPI_PASSWORD")

if not USERNAME or not PASSWORD:
    raise RuntimeError("INPI_USERNAME ou INPI_PASSWORD manquant dans le .env")

LOGIN_URL = "https://registre-national-entreprises.inpi.fr/api/sso/login"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
}

response = requests.post(LOGIN_URL, json=payload, timeout=10)
response.raise_for_status()

token = response.json()["token"]

HEADERS = {
    "Authorization": f"Bearer {token}"
}

#### Vérification de l'accès

In [3]:
print("Token OK :", token[:20], "...")

Token OK : eyJ0eXAiOiJKV1QiLCJh ...


#### Test

In [4]:
siren = "788653541"
url = f"https://registre-national-entreprises.inpi.fr/api/companies/{siren}/attachments"

# Récupérer et convertir directement en dict
attachments = requests.get(url, headers=HEADERS, timeout=10).json()

# Vérification rapide
if attachments:
    print("✅ Attachments OK :", list(attachments.keys()))
else:
    print("⚠️ Aucun résultat pour ce SIREN")

⚠️ Aucun résultat pour ce SIREN


In [5]:
# Exemple : récupération des bilans saisis
bilans_saisis = attachments.get("bilansSaisis", [])

print(f"Nombre de bilans saisis disponibles : {len(bilans_saisis)}")

Nombre de bilans saisis disponibles : 0


In [35]:
if bilans_saisis:  # Vérifie qu'il y a au moins un bilan
    bilan_id = bilans_saisis[0]["id"]
    url_bilan = f"https://registre-national-entreprises.inpi.fr/api/bilans-saisis/{bilan_id}"
    
    bilan_json = requests.get(url_bilan, headers=HEADERS, timeout=10).json()
    
    # Affichage formaté
    print("✅ Bilan récupéré :")
    print(json.dumps(bilan_json, indent=2, ensure_ascii=False))
else:
    print("⚠️ Aucun bilan saisi disponible pour ce SIREN")

✅ Bilan récupéré :
{
  "updatedAt": "2023-03-24T14:03:00+01:00",
  "siren": "552100554",
  "denomination": "PEUGEOT SA",
  "dateDepot": "2020-07-22",
  "numChrono": "21742",
  "confidentiality": "Public",
  "deleted": false,
  "id": "63eeeb67b40c7fba9219923e",
  "bilanSaisi": {
    "bilan": {
      "identite": {
        "siren": "552100554",
        "dateClotureExercice": "2019-12-31",
        "codeGreffe": "9201",
        "numDepot": "21742",
        "numGestion": "2017B04368",
        "codeActivite": "7010Z",
        "dateClotureExerciceNMoins1": "2018-12-31",
        "dureeExerciceN": "12",
        "dureeExerciceNMoins1": "00",
        "dateDepot": "2020-07-22",
        "codeSaisie": "01",
        "codeTypeBilan": "C",
        "codeDevise": "EUR",
        "codeOrigineDevise": "O",
        "codeConfidentialite": "0",
        "infoTraitement": "A1",
        "denomination": "PEUGEOT SA",
        "adresse": "92500 Rueil-Malmaison"
      },
      "detail": {
        "pages": [
          

In [ ]:
# Sauvegarde du bilan dans un fichier JSON
with open(f"bilan_{bilan_id}.json", "w", encoding="utf-8") as f:
    json.dump(bilan_json, f, ensure_ascii=False, indent=2)

print(f"Bilan sauvegardé : bilan_{bilan_id}.json")

Bilan sauvegardé : bilan_63eeeb67b40c7fba9219923c.json


In [13]:
bilan_saisi = attachments["bilansSaisis"][0]["bilanSaisi"]

identite = bilan_saisi["bilan"]["identite"]

print("SIREN :", identite["siren"])
print("Nom société :", identite["denomination"])
print("Adresse :", identite["adresse"])
print("Date de clôture :", identite["dateClotureExercice"])
print("Type de bilan :", identite["codeTypeBilan"])

SIREN : 552100554
Nom société : PEUGEOT SA
Adresse : 92500 Rueil-Malmaison
Date de clôture : 2019-12-31
Type de bilan : K


In [14]:
# Récupérer toutes les pages du bilan
pages = bilan_saisi["bilan"]["detail"]["pages"]

# Parcourir toutes les pages et toutes les liasses pour afficher les montants
for page in pages:
    print(f"--- Page {page['numero']} ---")
    for liasse in page["liasses"]:
        code = liasse["code"]
        m1 = liasse.get("m1")
        m2 = liasse.get("m2")
        m3 = liasse.get("m3")
        m4 = liasse.get("m4")
        print(code, m1, m2, m3, m4)

--- Page 1 ---
AJ   000010288000000 000009201000000
A1   000004312000000 000003608000000
AT   000016922000000 000014136000000
CS   000000719000000 000001072000000
A4   000002604000000 000002372000000
BJ   000038439000000 000033806000000
BT   000006269000000 000006710000000
BX   000002503000000 000001904000000
BZ   000003401000000 000003214000000
CJ   000031327000000 000028146000000
CO   000069766000000 000061952000000
--- Page 2 ---
DA 000000905000000 000000905000000  
P2 000018409000000 000016450000000  
DL 000021801000000 000019594000000  
P7 000002727000000 000002509000000  
DX 000014505000000 000013551000000  
EC 000079665000000 000072360000000  
EE 000069766000000 000061952000000  
--- Page 3 ---
FA   000074731000000 000074027000000
FJ   000074731000000 000074027000000
FR   000074731000000 000074027000000
GG   000004668000000 000004400000000
GP   000000192000000 000000188000000
GU   000000536000000 000000634000000
GV   -000000344000000 -000000446000000
GW   000004324000000 0000039

In [ ]:
total_actif = None
for page in pages:
    for liasse in page["liasses"]:
        if liasse["code"] == "AT":
            total_actif = liasse.get("m3")
            break

print("Total actif exercice N :", total_actif)

Total actif exercice N : 000016922000000


In [ ]:
pages = bilan_saisi["bilan"]["detail"]["pages"]

rows = []

# Parcourir toutes les pages et toutes les liasses
for page in pages:
    page_num = page["numero"]
    for liasse in page["liasses"]:
        rows.append({
            "page": page_num,
            "code": liasse["code"],
            "m1": liasse.get("m1"),
            "m2": liasse.get("m2"),
            "m3": liasse.get("m3"),
            "m4": liasse.get("m4")
        })

# Créer le DataFrame
df = pd.DataFrame(rows)

# Afficher un aperçu
print(df.head(20))

    page code               m1               m2               m3  \
0      1   AJ                                    000010288000000   
1      1   A1                                    000004312000000   
2      1   AT                                    000016922000000   
3      1   CS                                    000000719000000   
4      1   A4                                    000002604000000   
5      1   BJ                                    000038439000000   
6      1   BT                                    000006269000000   
7      1   BX                                    000002503000000   
8      1   BZ                                    000003401000000   
9      1   CJ                                    000031327000000   
10     1   CO                                    000069766000000   
11     2   DA  000000905000000  000000905000000                    
12     2   P2  000018409000000  000016450000000                    
13     2   DL  000021801000000  000019594000000 

In [17]:
# Remplacer les chaînes vides par 0 et convertir en int
for col in ["m1", "m2", "m3", "m4"]:
    df[col] = df[col].replace("", "0").astype(int)

print(df.head(20))

    page code           m1           m2           m3           m4
0      1   AJ            0            0  10288000000   9201000000
1      1   A1            0            0   4312000000   3608000000
2      1   AT            0            0  16922000000  14136000000
3      1   CS            0            0    719000000   1072000000
4      1   A4            0            0   2604000000   2372000000
5      1   BJ            0            0  38439000000  33806000000
6      1   BT            0            0   6269000000   6710000000
7      1   BX            0            0   2503000000   1904000000
8      1   BZ            0            0   3401000000   3214000000
9      1   CJ            0            0  31327000000  28146000000
10     1   CO            0            0  69766000000  61952000000
11     2   DA    905000000    905000000            0            0
12     2   P2  18409000000  16450000000            0            0
13     2   DL  21801000000  19594000000            0            0
14     2  

In [19]:
print("Codes disponibles dans ce bilan :", df["code"].unique())


Codes disponibles dans ce bilan : ['AJ' 'A1' 'AT' 'CS' 'A4' 'BJ' 'BT' 'BX' 'BZ' 'CJ' 'CO' 'DA' 'P2' 'DL'
 'P7' 'DX' 'EC' 'EE' 'FA' 'FJ' 'FR' 'GG' 'GP' 'GU' 'GV' 'GW' 'HK' 'R1'
 'R4' 'R5' 'R6' 'R8']


In [20]:
# Total actif exercice N
total_actif = df[df["code"] == "AT"]["m3"].iloc[0]
print("Total actif exercice N :", total_actif)

# Capitaux propres exercice N
if not df[df["code"] == "CS"].empty:
    capitaux_propres = df[df["code"] == "CS"]["m3"].iloc[0]
    print("Capitaux propres exercice N :", capitaux_propres)
else:
    print("Code CS introuvable dans ce bilan")

Total actif exercice N : 16922000000
Capitaux propres exercice N : 719000000


---

#### Connection BDD

In [32]:
# Charger .env
load_dotenv()

# Récupérer l'URL depuis les variables d'environnement
DATABASE_URL = os.getenv("NEON_DATABASE_URL")
if not DATABASE_URL:
    raise ValueError("La variable NEON_DATABASE_URL n'est pas définie dans .env")

# Connexion à la BDD
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()
print("✅ Connexion à Néon réussie !")

✅ Connexion à Néon réussie !


#### Création Tables BDD

In [ ]:
# Société
cur.execute("""
CREATE TABLE IF NOT EXISTS societes (
    siren VARCHAR PRIMARY KEY,
    denomination VARCHAR,
    adresse VARCHAR,
    date_cloture DATE,
    type_bilan VARCHAR,
    num_depot VARCHAR,
    code_activite VARCHAR
)
""")

# Bilan
cur.execute("""
CREATE TABLE IF NOT EXISTS lignes_bilan (
    id SERIAL PRIMARY KEY,
    siren VARCHAR REFERENCES societes(siren),
    page INT,
    code VARCHAR,
    libelle VARCHAR,
    m1 BIGINT,
    m2 BIGINT,
    m3 BIGINT,
    m4 BIGINT
)
""")

conn.commit()

#### Alimentation BDD

In [ ]:
# société
cur.execute("""
INSERT INTO societes (siren, denomination, adresse, date_cloture, type_bilan, num_depot, code_activite)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (siren) DO UPDATE
SET denomination = EXCLUDED.denomination,
    adresse = EXCLUDED.adresse,
    date_cloture = EXCLUDED.date_cloture,
    type_bilan = EXCLUDED.type_bilan
""", (
    identite["siren"],
    identite["denomination"],
    identite["adresse"],
    identite["dateClotureExercice"],
    identite["codeTypeBilan"],
    identite.get("numDepot"),
    identite.get("codeActivite")
))

conn.commit()

In [30]:
pages = bilan_saisi["bilan"]["detail"]["pages"]

# Exemple de mapping codes → libellés
code_map = {
    "AJ": "Immobilisations incorporelles",
    "A1": "Terrains",
    "AT": "Total Actif",
    "CS": "Capitaux propres",
    "CO": "Résultat net",
    "BJ": "Stocks",
    "BT": "Avances et acomptes",
    "BX": "Créances clients",
    "BZ": "Autres créances",
    "CJ": "Disponibilités",
    # ajouter le reste si nécessaire
}

for page in pages:
    page_num = page["numero"]
    for liasse in page["liasses"]:
        code = liasse["code"]
        libelle = code_map.get(code, "")
        m1 = int(liasse.get("m1") or 0)
        m2 = int(liasse.get("m2") or 0)
        m3 = int(liasse.get("m3") or 0)
        m4 = int(liasse.get("m4") or 0)

        cur.execute("""
        INSERT INTO lignes_bilan (siren, page, code, libelle, m1, m2, m3, m4)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            identite["siren"],
            page_num,
            code,
            libelle,
            m1, m2, m3, m4
        ))

conn.commit()

#### Vérification insertions

In [31]:
cur.execute("SELECT * FROM lignes_bilan WHERE siren = %s LIMIT 10", (identite["siren"],))
rows = cur.fetchall()
for row in rows:
    print(row)

(1, '552100554', 1, 'AJ', 'Immobilisations incorporelles', 0, 0, 10288000000, 9201000000)
(2, '552100554', 1, 'A1', 'Terrains', 0, 0, 4312000000, 3608000000)
(3, '552100554', 1, 'AT', 'Total Actif', 0, 0, 16922000000, 14136000000)
(4, '552100554', 1, 'CS', 'Capitaux propres', 0, 0, 719000000, 1072000000)
(5, '552100554', 1, 'A4', '', 0, 0, 2604000000, 2372000000)
(6, '552100554', 1, 'BJ', 'Stocks', 0, 0, 38439000000, 33806000000)
(7, '552100554', 1, 'BT', 'Avances et acomptes', 0, 0, 6269000000, 6710000000)
(8, '552100554', 1, 'BX', 'Créances clients', 0, 0, 2503000000, 1904000000)
(9, '552100554', 1, 'BZ', 'Autres créances', 0, 0, 3401000000, 3214000000)
(10, '552100554', 1, 'CJ', 'Disponibilités', 0, 0, 31327000000, 28146000000)
